## Data importing and manipulation

Packages used for this project:

In [ ]:
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import ttk
from functools import partial
from tkinter import messagebox

Importing, and loading data from csv files in dropbox into the notebook:

In [ ]:
vending_machine = pd.read_csv('https://www.dropbox.com/s/vy61ldio3t02lqy/vending_machine.csv?raw=1')
vending_machine['amount'].fillna(0, inplace=True)
vending_machine.head()

In [ ]:
stock = pd.read_csv('https://www.dropbox.com/s/qwl4i3slj4mklsj/stock.csv?raw=1')
stock.head()

#### Cleaning data:

In [ ]:
def column_split(snackname):
    if "'s" in snackname:
        for snackname in snackname.split("'s"):
            snackname = snackname.lstrip("'s ").title()
        return snackname
    else:
        for snackname in snackname.split("s' "):
            snackname = snackname.title()
        return snackname

stock['snack'] = stock['snack'].apply(column_split)
stock

In [ ]:
#dictionary to add the category automatically:
products = ['Diet Coke', 'Coke', 'Irn-bru', 'Mars', 'Milky Way', 'Sea Salt Crisps', 'Shortbread', 'Doritos']
categories = ['Drink', 'Drink','Drink','Chocolate','Chocolate','Crisp','Biscuits','Crisps']
prices = [3.75, 3.70, 3.00, 2.90, 2.85, 2.15, 2.50, 2.50]

product_info = {}
for pr, ca, pri in zip(products, categories, prices):
    product_info[pr]=[ca,pri]

Setting the machine code as index, and capitalize headings of columns:

In [ ]:
vm_woindex = pd.merge(vending_machine, stock, on='snack', how='left')
vm_woindex['Machine_Code']=vm_woindex['machine_code']
vm_woindex.set_index('Machine_Code', inplace=True)
vm_woindex.columns = map(str.upper, vm_woindex.columns)
vm_woindex.astype({'AMOUNT': 'int32'}).dtypes
vm = vm_woindex #I just shorten the name

#Rewrite the add_products functions [Just for the Irn-Brus]
def add_products(slot, snack_name, amount):
    vm.at[slot, 'SNACK'] = snack_name
    x = vm.at[slot, 'AMOUNT']
    vm.at[slot, 'AMOUNT'] = x+amount
    vm.at[slot,('CATEGORY','PRICE')]=product_info.get(snack_name)
    print('done!')
    print(x)

add_products('B4', 'Irn-bru', 3)
vm

## Vending Machine Simulation Program

In [ ]:
#setting the window interface:
window = Tk()
window.title('VENDING MACHINE 007')
window.geometry("450x600+50+50") #size of window
window.resizable(True,False)

#first window Frames:
buttonF = Frame(window)
buttonF.pack(side=TOP, fill=X, expand=False)

tableF = Frame(window)
tableF.pack(fill=X, expand=True)

exitF = Frame(window)
exitF.pack(side=BOTTOM, expand=False)


#counters:
count =0
count2= 0

# 1. function to prompt login_window:
def OpenLogin():
    #function to disable login after 4 attempts
    def prevent(event):
        global count
        count += 1
        if count==5:
            loginButton.config(state=DISABLED)
            loginButton.unbind("<Button-1>")
            login_window.destroy
        else:
            loginButton.config(state=NORMAL)
            attempts.config(text=f"Attempts: {count} out of 4")
            loginButton.bind("<Button-1>", prevent)
            
    #function to validate login:
    def validateLogin(): 
        userT = userE.get()
        passT = passE.get()
        correct = 0
        
        if userT == 'admin' and float(passT)<10:
            logged = True
            correct = 1
            messagebox.showinfo("update", "Login Succesful")
            loginButton.config(state=DISABLED)
            loginButton.unbind("<Button-1>")
            opt2.config(state=NORMAL) #once login, restock button will enable click
            login_window.destroy
                
        if correct == 0:
            messagebox.showwarning("update", "username or password incorrect please try again")
            
    #Log in window program:
    login_window = Toplevel(window)
    login_window.geometry("300x300")
    login_window.title("Login")
    login_menu=Frame(login_window)
    login_menu.pack(fill=X)
    bf = Frame(login_window)
    bf.pack(side=TOP, fill=X, expand=False)
    
    #dict with allowed username info:
    username_correct = {'username:':['admin', 'admin','admin', 'admin','admin', 'admin','admin', 'admin','admin'], 
                        'Password':['1', '2','3','4','5','6','7','8','9']}
    
    #labels and Entry for login:
    Label1 = Label(login_window, text='username')
    Label1.pack()
    userE = StringVar()
    Entry1 = Entry(login_window, textvariable=userE)
    Entry1.pack()
    
    Label2 = Label(login_window, text='password')
    Label2.pack()
    passE = StringVar()
    Entry2 = Entry(login_window, textvariable=passE)
    Entry2.pack()

    #login button
    loginButton = Button(login_window, text="login", command=validateLogin)
    loginButton.pack()
    loginButton.bind("<Button-1>", prevent)
    attempts = Label(login_window, text=f"Attempts: 0 out of 4")
    attempts.pack()

#Function to allow just one try on login
def justonelogin(event):
    global count2
    count2 += 1
    if count2==2:
        opt1.config(state=DISABLED)
        opt1.unbind("<Button-1>")
    else:
        opt1.config(state=NORMAL)
        opt1.bind("<Button-1>", justonelogin)
    
#login button from master menu:
opt1 = Button(buttonF, text ="Admin Login", bg = "light blue", command = OpenLogin, width=30, fg = "black")
opt1.pack()
opt1.bind("<Button-1>", justonelogin)

# 2. function to prompt restock window:
def OpenRestock():

    #restock window program:
    restock_window = Toplevel(window)
    restock_window.geometry("500x300")
    restock_window.title("Restock")
    
    restock_menu=Frame(restock_window)
    restock_menu.pack(fill=X)
    bf2 = Frame(restock_window)
    bf2.pack(fill=X, expand=True)
    
    product_cat = {'Drink':['Diet Coke','Coke','Irn-bru'], 'Chocolate':['Mars','Milky Way'], 
                   'Crisps':['Sea Salt Crisps','Doritos'],'Biscuits':['Shortbread', 'None']}

    slot_product= {'Diet Coke': ['A1', 'A2'], 'Coke': ['A3'], 'Irn-bru': ['A4', 'B4'], 'Mars': ['B1', 'B2'], 
               'Milky Way': ['B3', 'C4'], 'Sea Salt Crisps': ['C1', 'C2'], 'Shortbread': ['C3', 'D4'], 
               'Doritos': ['D1', 'D2']}
    
    #Frame for selection:
    selection_stock = LabelFrame(restock_menu, text="Select Snack and Slot to Restock", width=10, height=10)
    selection_stock.pack(side = TOP, expand=True)
    
    #Menu to select category to restock
    CatB = Menubutton(selection_stock, text="Category", relief='raised')
    CatB.pack(side= LEFT)
    cat_selected = StringVar()
    menu_cat=Menu(CatB, tearoff=0)
    
    #Label that shows the category selected:
    CatL = Label(selection_stock, bg="white", width=8, height=2)
    CatL.pack(side= LEFT)
    
    #Menu to select snack based in category:
    SnaB = Menubutton(selection_stock, text='Product', relief='raised', state=DISABLED)
    SnaB.pack(side= LEFT)
    Sna_selected = StringVar()
    menu_sna = Menu(SnaB, tearoff=0)
    
    #Label that shows the Snack selected:
    SnaL = Label(selection_stock, bg="white", width=8, height=2)
    SnaL.pack(side=LEFT)
    
    #Menu Button to select slot based in snack:
    SloB = Menubutton(selection_stock, text='Slot',  relief='raised', state=DISABLED)
    SloB.pack(side=LEFT)
    Slo_selected =StringVar()
    menu_slo = Menu(SloB, tearoff=0)
    
    #Label that shows the Slot selected:
    SloL = Label(selection_stock, bg='white', width=8, height=2)
    SloL.pack(side=LEFT)

    
    #Menu to save info to restock in vending machine:
    def add_products(slot, snack_name):
        if vm.at[slot, 'AMOUNT'] == 8:
            messagebox.showwarning("Restock", f'{slot} is full')
            restock_menu.destroy
            
        elif vm.at[slot, 'AMOUNT'] + 1 > 8:
            messagebox.showwarning("Restock", 'Not enough space, please lower amount to restock')
            
        elif vm.at[slot, 'AMOUNT']+ 1 <=8:
            for i in vm_astable.get_children():
                vm_astable.delete(i)
            vm.at[slot, 'AMOUNT'] = vm.at[slot, 'AMOUNT'] + 1
            vm.at[slot, 'SNACK'] = snack_name
            vm.at[slot,('CATEGORY','PRICE')]=product_info.get(snack_name)
            messagebox.showinfo("Restock", 'Restock succesful')
            draw_table()
            SaveB.config(state=DISABLED)
        
    def Save_func():
        ff = Slo_selected.get()
        bb = Sna_selected.get()
        add_products(ff, bb)
        
    
    SaveB = Button(bf2, text="Save", relief='raised', command=Save_func, bg="light blue", width=10)
    SaveB.pack()
        
    #Function to change labels:
    def c_CatL():
        CatL.config(text=cat_selected.get())
        SnaB.config(state=NORMAL)
        out1 = cat_selected.get()
        
        def get_menu_for_snack(ii): 
            if out1 == ii:
                for sna in product_cat[ii]:
                    menu_sna.add_radiobutton(label=sna, value=sna, variable=Sna_selected, command=c_SnaL)
                    SnaB['menu']=menu_sna
                    
        get_menu_for_snack(out1)
        CatB.config(state=DISABLED)
        
    for cat in product_cat.keys():
        menu_cat.add_radiobutton(label=cat, value=cat, variable=cat_selected, command=c_CatL)
    
    def c_SloL():
        SloL.config(text=Slo_selected.get())
        SloB.config(state=DISABLED)
        
    def c_SnaL():
        SnaL.config(text=Sna_selected.get())
        SloB.config(state=NORMAL)
        out2 = Sna_selected.get()
        
        def get_menu_for_slots(mm):
            if out2==mm:
                for slo in slot_product[mm]:
                    menu_slo.add_radiobutton(label=slo, value=slo, variable =Slo_selected, command=c_SloL)
                    SloB['menu']=menu_slo
                    
        get_menu_for_slots(out2)
        SnaB.config(state=DISABLED)
        
    CatB['menu']=menu_cat
    
opt2 = Button(buttonF, 
              text= "Restock Products", 
              bg = "light blue",
              command=OpenRestock,
              width=30,
              fg = "black",
              state=DISABLED)
opt2.pack()

#3 Menu to buy products:
def BuyBuy():
    buy_window = Toplevel(window)
    buy_window.geometry("410x200")
    buy_window.title("Buy products")
    
    Slo_tobuy = Menubutton(buy_window, text='Select slot', relief='raised')
    Slo_tobuy.place(x=60, y=10)
    Slot_tobuy= StringVar()
    menu_slottobuy = Menu(Slo_tobuy, tearoff=0)
    
    Message_slot = Label(buy_window, text='Select a Slot, please', bg='white', width=30)
    Message_slot.place(x=200, y=10)
    
    Message_topay = Label(buy_window, text=' ', bg='White', width=50)
    Message_topay.place(x=30, y=60)
            
    def isDigit(xxx):
        if str(xxx) == xxx:
            return False
        if float(xxx) == xxx:
            return True
        else:
            return False
    
    def transaction():
        Slo_tobuy.config(state=NORMAL)
        bling2=money.get()
        if isDigit(bling2) == True:
            sloty=Slot_tobuy.get()
            bling=float(money.get())
            if bling > vm.at[sloty,'PRICE']:
                toreturn= round(bling-vm.at[sloty,'PRICE'], 2)
                messagebox.showinfo('Dispatched', f"Thank you for your purchase, your change is {toreturn}")
                vm.at[sloty,'AMOUNT'] = vm.at[sloty,'AMOUNT']-1
                for i in vm_astable.get_children():
                    vm_astable.delete(i)
                draw_table()
            elif bling == vm.at[sloty,'PRICE']:
                messagebox.showinfo('Dispatched', "Thank you for your purchase")
                vm.at[sloty,'AMOUNT'] = vm.at[sloty,'AMOUNT']-1
                for i in vm_astable.get_children():
                    vm_astable.delete(i)
                draw_table()
            else:
                messagebox.showwarning("update", "Please insert more money")            
        else:
            messagebox.showwarning("update", "Please enter a valid amount $")
    
    money=StringVar()
    Money_toinsert = Entry(buy_window, bg='white', textvariable=money)
    Money_toinsert.place(x=60, y=100)
    
    Money_insert= Button(buy_window, text='Insert Money', relief='raised', command=transaction, state=DISABLED)
    Money_insert.place(x=200,y=100)
    
    
    def change_message():
        bb=Slot_tobuy.get()
        Message_slot.config(text=f"You've selected slot {bb}")
        vv = vm.at[bb, 'PRICE']
        if vv>0:
            Message_topay.config(text=f"Please, insert ${vv}")
            Money_insert.config(state=NORMAL)
            Money_toinsert.config(state=NORMAL)
        else:
            Message_topay.config(text=f"This Slot is Empty")
            Money_insert.config(state=DISABLED)
    
    for slotb in vm['MACHINE_CODE']:
        menu_slottobuy.add_radiobutton(label=slotb, value=slotb, command=change_message, variable=Slot_tobuy)
        Slo_tobuy['menu']=menu_slottobuy

opt3 = Button(buttonF, 
              text="Buy Products", 
              command=BuyBuy,
              width=30,
              bg = "White",
              fg = "black")
opt3.pack()

#Show the vending machine as a table:
vm_astable=ttk.Treeview(tableF, height=20)

def draw_table():
    vm_astable['column']= list(vm.columns)
    vm_astable['show']= 'headings'

    for column in vm_astable['column']:
        vm_astable.heading(column, text=column)
        
    vm_astable.column("# 1", anchor='sw', width=80)
    vm_astable.column("# 2", anchor='sw', width=100)
    vm_astable.column("# 3", anchor='sw', width=80)
    vm_astable.column("# 4", anchor='sw', width=100)
    vm_astable.column("# 5", anchor='sw', width=50)

    #put data in treeview:
    df_rows = vm.to_numpy().tolist()

    for row in df_rows:
        vm_astable.insert('', "end", values=row)
    #pack
    vm_astable.pack()

draw_table()
    
    
opt4 = Button(exitF,  
              text="Exit", 
              command=window.destroy,
              width=10,
              bg = "Black",
              fg = "White")
opt4.pack()

#deploying window:
window.mainloop()